# Técnica de instruction tuning

En este cuaderno se mostrará como realizar un instruction tunning a un gran modelo de lenguaje (LLM), concretamente, al modelo [FLAN-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5) creado por Google.

Mediante esta técnica, se puede ajustar un modelo a una tarea específica instruyéndolo con prompts, es decir, con conjuntos de instrucciones en lenguaje natural que guían al modelo en la realización de la tarea. En este caso, se utilizará un corpus en inglés de clasificación de noticias en diferentes categorías.

## Paso 1: Instalación e importación de librerías y definición de parámetros

Se instalan a continuación las librerías necesarias para la ejecución de este cuaderno.

In [1]:
%%capture

!pip install transformers==4.27.2
!pip install datasets==2.15.0
!pip install tqdm==4.66.1
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install wandb
!pip install accelerate

Se importan las librerías necesarias para este cuaderno.

In [2]:
from transformers import T5ForConditionalGeneration, AutoTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, Dataset
from tqdm import tqdm

Una vez importadas, se definen una serie de parámetros que se utilizarán a lo largo del cuaderno.

- **model_name**: Nombre del modelo que será reentrenado.
- **dataset_name**: Nombre del dataset utilizado.
- **epochs**: Número de épocas de entrenamiento.
- **max_length**: Longitud máxima de las secuencias de entrada. El máximo del modelo son 512.
- **use_cuda**: Booleano que indica si se utilizará la GPU para entrenar.
- **results_dir**: Directorio donde se guardarán los resultados.

In [3]:
args = {}
args['model_name'] = "google/flan-t5-small"
args['dataset_name'] = "fancyzhx/ag_news"
args['epochs'] = 2
args['max_length'] = 400
args['use_cuda'] = True
args['results_dir'] = "experiments/test"

## Paso 2: Carga y procesado del corpus

Mediante la función `load_dataset` de la librería `datasets`, se puede cargar un dataset de manera sencilla que esté publicado en HuggingFace.

En este caso se va cargar el dataset `fancyzhx/ag_news` que contiene más de 100.000 noticias en inglés clasificadase en 4 clsaes: Sci/Tech, Sports, Business y World. Toda la información se encuentra en [este enlace](https://huggingface.co/datasets/fancyzhx/ag_news).

In [4]:
dataset = load_dataset(args['dataset_name'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Las columnas que contiene el dataset son las siguientes:

- **text**: Texto de la noticia.
- **label**: Clase de la noticia (0, 1, 2 o 3).

Se define la función `add_label_column` que se encargará de transformar el número de la clase en una etiqueta legible (Sci/Tech, Sports, Business o World). Para ello, se definen los diccionarios `label2name` y `name2label` que mapean las clases con las etiquetas.

In [5]:
label2id = {
    'World': 0,
    'Sports': 1,
    'Business': 2,
    'Sci/Tech': 3
}

id2label = {v: k for k, v in label2id.items()}

In [6]:
#Append a column in the dataset for the labelText
def add_label_column(example):
    example['labelText'] = id2label[example['label']]
    return example

dataset = dataset.map(add_label_column)

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

Veamos un ejemplo del dataset

In [7]:
print("Texto: ", dataset['train'][0]['text'])
print("Categoria (en texto): ", dataset['train'][0]['labelText'])
print("Categoria (en Id): ", dataset['train'][0]['label'])

Texto:  Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
Categoria (en texto):  Business
Categoria (en Id):  2


Vemos que por cada ejemplo, tenemos un texto que corresponde a una breve noticia/titular, la cual está en este caso, incluida en la categoria Business (la cual tiene el id 2)

Este dataset contiene los datos divididos en 2 particiones: `train` y `test`. Como originalmente no contiene una partición de validación, se creará una extrayendo 3.00 muestras del conjunto de entrenamiento. Se utilizará la partición de `train` para entrenar el modelo, la de `validation` para validar el modelo y la de `test` para evaluar el modelo.

In [8]:
#Extract 3000 sample from the train set to create the validation set
newSplits = dataset['train'].train_test_split(test_size=3000, shuffle=True)

dataset['train'] = newSplits['train']
dataset['validation'] = newSplits['test']

Para agilizar el proceso de entrenamiento, se reducirá el tamaño del dataset a 10.000 muestras para la partición de `train` y a 3.000 muestras para las particiones de `validation` y `test`.

In [9]:
#Get just 10.000 samples for training, 3.000 for validation and 3.000 for testing
dataset['train'] = dataset['train'].select(range(10000))
dataset['validation'] = dataset['validation'].select(range(3000))
dataset['test'] = dataset['test'].select(range(3000))

Una vez tenemos los datos cargados y correctos, se ha definido la función `add_prompt_column`, la cual se encargará de añadir una columna al corpus con el prompt correspondiente a cada noticia. Para ello, se ha definido el siguiente prompt:

```
Given the following text: {text}
Predict its corresponding category (World, Sports, Business, Sci/Tech):
```

Se mapeará cada noticia con su prompt correspondiente y se guardará en una nueva columna llamada `prompt`.

In [10]:
#Create the prompt
def add_prompt_column(example):
    #Replace \ by a space
    example['text'] = example['text'].replace('\\', ' ')
    example['prompt'] = f'Given the following text:{example["text"]}\nPredict its corresponding category (World, Sports, Business, Sci/Tech):'
    return example

dataset = dataset.map(add_prompt_column)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

## Paso 3: Reentrenamiento del modelo

Se define la función `preprocess_function` que se encargará de procesar los datos de entrada para que puedan ser utilizados por el modelo.

Esta función, usando el tokenizador, se tokenizan los prompts. Además, se tokenizan también los textos correspondientes a las categorias de cada una de las noticias.

<span style="color:red">Atención:</span> Esta función está definida para poder ser usada de forma paralela (`batched=True`) al ser usada por la función `map` de la librería `datasets`. Esto permite que el procesado de los datos sea más rápido, al procesar varios elementos a la vez. Es por ello que te pueda parecer que la función está definida de forma extraña.

In [11]:
def preprocess_function(sample, padding="max_length"):

    # Usar el tokenizador para generar los inputs del modelo para cada elemento del dataset, dado el texto
    model_inputs = tokenizer(
        sample['prompt'],
        max_length=args['max_length'],
        padding=padding,
        truncation=True,
    )

    # Usar el tokenizador para generar los targets del modelo para cada elemento del dataset, dado el texto anotado
    target_diseases = tokenizer(
        text_target=sample["labelText"],
        max_length=args['max_length'],
        padding=padding,
        truncation=True,
    )

    # Si se está haciendo padding (es decir, si se está fijando el tamaño máximo de la secuencia de entrada y no considerando el
    # tamaño de cada secuencia de entrada), se reemplazan todos los tokens de padding por -100 para que no se consideren en la
    # función de pérdida. Esto se realiza para que el modelo ignore los tokens de padding en la función de pérdida y no se
    # penalice por ellos, ni se aprenda a predecirlos.
    if padding == "max_length":
        target_diseases["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label]
            for label in target_diseases["input_ids"]
        ]

    # Se agregan los targets al diccionario de inputs del modelo
    model_inputs["labels"] = target_diseases["input_ids"]

    # Se retorna el diccionario de inputs del modelo
    return model_inputs

Se importa el tokenizador y el modelo.

El tokenizador se importa haciendo uso de la función `AutoTokenizer` de la librería `transformers`. Esta función se encarga de cargar el tokenizador adecuado para el modelo que se le pasa como parámetro.

El modelo se carga haciendo uso de la función `T5ForConditionalGeneration` de la librería `transformers`. Esta función se encarga de cargar el modelo T5 para fine-tuning, especificando que la tarea a realizar es la de generación condicional. (Más información en [este enlace](https://huggingface.co/docs/transformers/model_doc/t5#transformers.T5ForConditionalGeneration))

In [12]:
tokenizer = AutoTokenizer.from_pretrained(args['model_name'])
model = T5ForConditionalGeneration.from_pretrained(args['model_name'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Antes de utilizar la función definida, debemos de dilucidar cuál es el tamaño máximo de los textos de las noticias. Para ello, iteraremos sobre el dataset y guardaremos el tamaño máximo de los textos en la variable `max_text_length`. Ajustando el tamaño máximo de los textos, podremos reducir el tiempo de procesado de los datos.

In [13]:
#Calculate the maximum length of the input and output
baseText = 'Given the text: "' + \
        '", the corresponding category (World, Sports, Business, Sci/Tech) is: '

#Get the text with the maximum length
maxText = dataset['train']['text'][0]
for text in dataset['train']['text']:
    if len(text) > len(maxText):
        maxText = text

#tokenize the text
inputs = tokenizer(
    baseText + maxText,
    max_length=203030,
    truncation=True,
)

#Get the maximum length of the input
max_input_length = len(inputs['input_ids'])

print("Max input length: ", max_input_length)

Max input length:  389


Una vez se tiene el tokenizador y la función `preprocess_function` definidos, se aplica esa función sobre todo el dataset mediante la función `map` de la librería `datasets`.

In [14]:
tokenized_dataset = dataset.map(
    preprocess_function, batched=True, remove_columns=["text", "labelText", "label", "prompt"]
)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Se indica si se va a utilizar la GPU para entrenar el modelo.

In [15]:
if args['use_cuda']:
    model.cuda()

Se definen los hiperparámetros de entrenamiento utilizando la clase `Seq2SeqTrainingArguments` de la librería `transformers`. Esta clase permite definir los hiperparámetros de entrenamiento de una manera sencilla. (Más información en [este enlace](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments)).

En este caso se han definido los siguientes hiperparámetros:

- **do_train**: Booleano que indica si se va a entrenar el modelo. En este caso se va a entrenar el modelo.
- **do_eval**: Booleano que indica si se va a evaluar el modelo usando el dataset de validación. En este caso se va a evaluar el modelo.
- **evaluation_strategy**: Estrategia de evaluación. En este caso se evaluará al final de cada época.
- **logging_strategy**: Estrategia de logging (mostrar mensajes). En este caso se hará logging al final de cada época.
- **save_strategy**: Estrategia de guardado. En este caso no se guardará el modelo al final de cada época, sino que se guardará el mejor modelo.
- **per_device_train_batch_size**: Tamaño del batch de entrenamiento por dispositivo.
- **per_device_eval_batch_size**: Tamaño del batch de evaluación por dispositivo.
- **auto_find_batch_size**: Booleano que indica si se va a buscar el tamaño de batch más grande posible para entrenar el modelo. En este caso se va a buscará.
- **gradient_accumulation_steps**: Número de pasos de acumulación de gradientes. Esto se suele utilizar al entrenar modelos grandes, los cuales requieren de mucha memoria. En este caso se acumulará un paso de gradiente.
- **learning_rate**: Tasa de aprendizaje. En este caso se utilizará una tasa de aprendizaje de 1e-5.
- **num_train_epochs**: Número de épocas de entrenamiento.
- **output_dir**: Directorio donde se guardarán los resultados.
- **logging_dir**: Directorio donde se guardarán los logs.

In [16]:
training_args = Seq2SeqTrainingArguments(
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="no",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    auto_find_batch_size=True,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    num_train_epochs=args['epochs'],
    output_dir=args['results_dir'],
    logging_dir=f"{args['results_dir']}/logs",
)

Se define ahora el _data collator_, que es el encargado de procesar los datos de entrada para que puedan ser utilizados por el modelo. En este caso se utiliza la clase `DataCollatorForSeq2Seq` de la librería `transformers`. Esta clase se encarga de procesar los datos de entrada para que puedan ser utilizados por el modelo T5. (Más información en [este enlace](https://huggingface.co/docs/transformers/main_classes/data_collator#transformers.DataCollatorForSeq2Seq)).

Como se quieren ignorar los tokens de padding, en el parámetro `label_pad_token_id` se indica el valor -100 para que se ignoren. Ignorando los tokens de padding, se consigue que el modelo no tenga en cuenta los tokens de padding a la hora de calcular la pérdida.

Respecto al valor de `pad_to_multiple_of`, se indica el valor 8 para que el tamaño de las secuencias de entrada sea múltiplo de 8. Esto es necesario para poder utilizar la GPU para entrenar el modelo.

In [17]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=32,
)

Ahora, se define el objeto para reentrenar el modelo, en este caso se utiliza la clase `Seq2SeqTrainer` de la librería `transformers`. Esta clase se encarga de reentrenar un modelo T5. (Más información en [este enlace](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainer)).

En él se define el modelo a usar, los argumentos de entrenamiento, el _data collator_ y el dataset de entrenamiento y de validación.

Además, mediante la orden `model.config.use_cache = False` se indica que no se utilice la caché del modelo. Haciendo esto, se consigue que el modelo no utilice la caché de las capas de atención que ya conocía, por tanto, los parámetros del modelo se actualizarán en base a los nuevos datos.

In [18]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator
)

model.config.use_cache = False


Se ejecuta el reentrenamiento del modelo.

La pérdida se calcula utilizando la función de entropía cruzada [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html).

<span style="color:red">Atención:</span> Puede ser que te pida una API Key de WandB. WandB (Weights & Biases) es una plataforma integral para el seguimiento y visualización de experimentos de aprendizaje automático. Facilita el registro y comparación de hiperparámetros y métricas, así como la colaboración entre equipos al proporcionar un espacio centralizado para compartir resultados y códigos. Con integración fácil en bibliotecas populares, WandB se destaca por su capacidad para mejorar la eficiencia en la gestión y comprensión de modelos, convirtiéndose en una herramienta valiosa para profesionales de aprendizaje automático.

In [19]:
modelTrainer = trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: egr68 (gplsi_continual) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.213400,0.136217
2,0.187100,0.136217


Una vez reentrenado, se guarda el modelo y el tokenizador en el directorio `results_dir`.

In [20]:
trainer.model.save_pretrained(args['results_dir'])
tokenizer.save_pretrained(args['results_dir'])

('experiments/test/tokenizer_config.json',
 'experiments/test/special_tokens_map.json',
 'experiments/test/spiece.model',
 'experiments/test/added_tokens.json',
 'experiments/test/tokenizer.json')

## Paso 4: Inferencia

Para realizar la inferencia, se utiliza el modelo ya reentrenado y el tokenizador. Se importan a continuación.

In [21]:
model = T5ForConditionalGeneration.from_pretrained(args['results_dir'])
tokenizer = AutoTokenizer.from_pretrained(args['results_dir'])

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:415: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

Se crea una función auxiliar llamada `generateOutput`, en la que dado el modelo, el tokenizador y un prompt, se genera la salida del modelo.

In [22]:
def generateOutput(model, tokenizer, prompt):

    #Se tokeniza el prompt
    inputs = tokenizer(prompt, return_tensors="pt")

    #Se obtiene la salida del modelo, dados:
    # - los inputs tokenizados
    # - la máscara de atención
    # - la longitud máxima de la secuencia de salida, establecida en este caso en 32
    # - el número de beams a usar en la decodificación, establecido en este caso en 5. Un beam es una hipótesis de salida
    #   que el modelo considera como una posible solución al problema. El modelo genera varias hipótesis de salida y
    #   selecciona la mejor de ellas como la salida final.
    # - early_stopping=True para que el modelo deje de generar hipótesis de salida cuando todas las hipótesis generadas
    #   tengan el token de fin de secuencia (</s>) o cuando se haya generado el número máximo de hipótesis de salida
    outputs = model.generate(
        inputs.input_ids.to(model.device),
        attention_mask=inputs.attention_mask.to(model.device),
        max_length=50,
        num_beams=5,
        early_stopping=True,
        return_dict_in_generate=True
    )
    decoded_output = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)

    return decoded_output

Se puede comprobar con el siguiente código que la inferencia funciona correctamente en un único ejemplo. Puedes probar con otros ejemplos cambiando el valor de i.

Para estos ejemplos, hemos utilizado el conjunto de entrenamiento, por tanto, es normal que el modelo suela acertar la categoría de la noticia.

In [23]:
i = 4
generated = generateOutput(model, tokenizer, dataset['train'][i]['prompt'])
real = dataset['train'][i]['labelText']

print(f"Prompt: {dataset['train'][i]['prompt']}")
print(f"Generated: {generated}")
print(f"Real: {real}")
print(f"Is the generated text the same as the real one? Yes" if generated == real else "Is the generated text the same as the real one? No")

/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


Prompt: Given the following text:Climate change already affecting the global environment, two &lt;b&gt;...&lt;/b&gt; Global warming has had little noticeable impact in Washington, DC Politicians in the nation #39;s capital have been reluctant to set limits on the carbon dioxide 
Predict its corresponding category (World, Sports, Business, Sci/Tech):
Generated: Business
Real: Sci/Tech
Is the generated text the same as the real one? No


Se define una función llamada `getMatchMetricBatch` para que realice una métrica que simplemente mida las coincidencias exactas entre la salida del modelo y la categoría real con respecto al total.

Para agilizar el proceso, se realizará la inferencia por lotes (batch). Para ello, se define la función `generateOutputBatch`, la cual se encargará de generar la salida del modelo para un lote de ejemplos.

In [24]:
def generateOutputBatch(model, tokenizer, prompts, batch_size=32):

    outputs = []
    for i in range(0, len(prompts), batch_size):
        # Select a batch of prompts
        batch_prompts = prompts[i:i + batch_size]

        # Tokenize the batch
        inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True)

        # Move tensors to the same device as the model
        inputs = {key: tensor.to(model.device) for key, tensor in inputs.items()}

        # Generate outputs for the batch
        batch_outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=50,
            num_beams=5,
            early_stopping=True,
            return_dict_in_generate=True
        )

        # Decode the generated sequences and append them to the results
        decoded_outputs = [tokenizer.decode(seq, skip_special_tokens=True) for seq in batch_outputs.sequences]
        outputs.extend(decoded_outputs)

    return outputs

In [25]:
def getMatchMetricBatch(model, tokenizer, split, batch_size=32):
    matches = 0
    total = len(split)

    # Process the dataset in batches
    for i in tqdm(range(0, total, batch_size)):
        # Extract a batch of prompts and labels
        batch = split[i:i + batch_size]
        prompts = batch['prompt']
        true_labels = batch['labelText']

        # Generate outputs for the batch
        generated_outputs = generateOutputBatch(model, tokenizer, prompts, batch_size=batch_size)

        # Compare generated outputs with true labels
        for generated, true_label in zip(generated_outputs, true_labels):
            if generated.strip() == true_label.strip():
                matches += 1

    # Calculate accuracy
    accuracy = matches / total

    return {'matches': matches, 'total': total, 'accuracy': accuracy}

Se ejecutan las métricas y se muestran los resultados.

In [26]:
batch_size = 12

print("Evaluating training set...")
trainMetric = getMatchMetricBatch(model, tokenizer, dataset['train'], batch_size=batch_size)
print(f"Train matches: {trainMetric['matches']}")
print(f"Train accuracy: {trainMetric['accuracy']}")

print("Evaluating validation set...")
validationMetric = getMatchMetricBatch(model, tokenizer, dataset['validation'], batch_size=batch_size)
print(f"Validation matches: {validationMetric['matches']}")
print(f"Validation accuracy: {validationMetric['accuracy']}")

print("Evaluating test set...")
testMetric = getMatchMetricBatch(model, tokenizer, dataset['test'], batch_size=batch_size)
print(f"Test matches: {testMetric['matches']}")
print(f"Test accuracy: {testMetric['accuracy']}")

Evaluating training set...


100%|██████████| 834/834 [54:05<00:00,  3.89s/it]


Train matches: 8876
Train accuracy: 0.8876
Evaluating validation set...


100%|██████████| 250/250 [16:47<00:00,  4.03s/it]


Validation matches: 2645
Validation accuracy: 0.8816666666666667
Evaluating test set...


100%|██████████| 250/250 [16:04<00:00,  3.86s/it]

Test matches: 2634
Test accuracy: 0.878


Se puede observar que el modelo ha acertado en la mayoría de los casos, superando en todos los conjuntos el 80% de aciertos.